In [2]:
# !pip install open3d
import open3d as o3d
import numpy as np
import sys

sys.path.append('./data/')
from camera import *
from dataloader import *
from trajprocessor import *

sys.path.append('./algorithm/')
from measurement import *
from pose import *
from update import *
from prediction import *

sys.path.append('./eval')
import associate

from pipeline import *

import matplotlib.pyplot as plt
import time

In [4]:
data_folders = ["./data/datasets/" + "rgbd_dataset_freiburg1_360/", # 0
                    "./data/datasets/" + "rgbd_dataset_freiburg1_plant/", # 1
                    "./data/datasets/" + "rgbd_dataset_freiburg1_xyz/", # 2
                    "./data/datasets/" + "rgbd_dataset_freiburg2_coke/", # 3
                    "./data/datasets/" + "rgbd_dataset_freiburg2_dishes/", # 4
                    "./data/datasets/" + "rgbd_dataset_freiburg2_flowerbouquet/", # 5
                    "./data/datasets/" + "rgbd_dataset_freiburg2_flowerbouquet_brownbackground/", # 6
                    "./data/datasets/" + "rgbd_dataset_freiburg2_metallic_sphere/", # 7
                    "./data/datasets/" + "rgbd_dataset_freiburg2_metallic_sphere2/", # 8
                    "./data/datasets/" + "rgbd_dataset_freiburg2_xyz/", # 9
                    "./data/datasets/" + "rgbd_dataset_freiburg3_cabinet/", # 10
                    "./data/datasets/" + "rgbd_dataset_freiburg3_nostructure_notexture_far/", # 11
                    "./data/datasets/" + "rgbd_dataset_freiburg3_sitting_static/", # 12
                    "./data/datasets/" + "rgbd_dataset_freiburg3_sitting_xyz/", # 13
                    "./data/datasets/" + "rgbd_dataset_freiburg3_structure_notexture_far/", # 14
                    "./data/datasets/" + "rgbd_dataset_freiburg3_teddy/", # 15
                    "./data/datasets/" + "rgbd_dataset_freiburg3_walking_static/"] # 16

data_folder = data_folders[10]
filtered_depths = True

kf = KinectFusion(data_folder=data_folder, depth_filtered=filtered_depths)

KinectFusion pipeline total frame number: 1121
Depth filtered: True


In [14]:
for i in range(len(kf.timestamps)):
    timestamp = kf.timestamps[i]
    success, gt_pose = interpolate_pose(kf.gt_traj, kf.gt_timestamps, timestamp)
    if success:
        extrinsic = trajectory_to_matrix(gt_pose)
        print(i, success, extrinsic)

Timestamp out of range
Timestamp out of range
Timestamp out of range
3 True [[ 9.99999996e-01  1.29463429e-08  8.92579413e-05  5.57943099e-05]
 [ 1.29463429e-08  9.99999958e-01 -2.90088309e-04  2.23177220e-04]
 [-8.92579413e-05  2.90088309e-04  9.99999954e-01  5.97610200e-08]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  1.00000000e+00]]
4 True [[ 9.99998146e-01  4.77030687e-04  1.86561967e-03  1.25692124e-03]
 [-4.68387770e-04  9.99989170e-01 -4.63042897e-03  6.53632823e-03]
 [-1.86780832e-03  4.62954655e-03  9.99987539e-01  4.27926399e-04]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  1.00000000e+00]]
5 True [[ 9.99972255e-01  4.18500967e-03  6.16245959e-03  3.27453974e-03]
 [-4.11183668e-03  9.99921460e-01 -1.18391679e-02  1.28560032e-02]
 [-6.21152262e-03  1.18135004e-02  9.99910925e-01  5.13674522e-04]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  1.00000000e+00]]
6 True [[ 9.99965006e-01  4.98901334e-03  6.71534339e-03  5.42644078e-03]
 [-4.91129840e-03  9.99921352

596 True [[ 0.65983162 -0.7421829   0.11741709  2.4892878 ]
 [-0.60049974 -0.6147652  -0.51133533  0.19734633]
 [ 0.45168827  0.26688629 -0.85132216  0.23075527]
 [ 0.          0.          0.          1.        ]]
597 True [[ 0.65805571 -0.74413876  0.1149791   2.48591905]
 [-0.60369381 -0.61267234 -0.51008469  0.19472142]
 [ 0.4500183   0.26625197 -0.85240449  0.22540936]
 [ 0.          0.          0.          1.        ]]
598 True [[ 0.65641478 -0.74584088  0.11331825  2.48197141]
 [-0.60503138 -0.61019701 -0.51146518  0.19524017]
 [ 0.4506181   0.26717221 -0.85179947  0.22117086]
 [ 0.          0.          0.          1.        ]]
599 True [[ 0.6539372  -0.74756849  0.11622177  2.47697306]
 [-0.60536877 -0.60918813 -0.51226796  0.19873252]
 [ 0.45375631  0.26463405 -0.8509254   0.22826788]
 [ 0.          0.          0.          1.        ]]
600 True [[ 0.65179895 -0.74973184  0.11428166  2.47221236]
 [-0.60789099 -0.60658628 -0.51236865  0.19642122]
 [ 0.45346078  0.26449056 -0.8511

In [3]:
sys.path.append('./eval')
import associate

gt_path = './data/datasets/rgbd_dataset_freiburg1_xyz/groundtruth.txt'
gt_list = associate.read_file_list(gt_path)

timestamps = list(gt_list.keys())
timestamps.sort()

for timestamp in timestamps:
    gt_list[timestamp] = convert_line_to_floats(gt_list[timestamp])

delta = np.array([0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0]) - gt_list[timestamps[0]]
for timestamp in timestamps:
    gt_list[timestamp] = gt_list[timestamp] + delta
    print(gt_list[timestamp])
    

[0. 0. 0. 0. 0. 0. 1.]
[-2.0000e-03  1.0000e-04 -2.0000e-03 -3.0000e-04  4.0000e-04 -5.0000e-04
  1.0006e+00]
[-3.800e-03  1.000e-04 -4.100e-03  4.000e-04  9.000e-04 -1.000e-04
  1.002e+00]
[-6.1000e-03  1.0000e-04 -6.2000e-03  7.0000e-04  1.0000e-03 -1.0000e-04
  1.0027e+00]
[-8.1000e-03  3.0000e-04 -8.2000e-03  1.6000e-03  1.6000e-03  1.0000e-03
  1.0041e+00]
[-1.0100e-02  3.0000e-04 -1.0500e-02  1.6000e-03  1.4000e-03  4.0000e-04
  1.0043e+00]
[-1.2400e-02  3.0000e-04 -1.2700e-02  1.9000e-03  1.5000e-03  2.0000e-04
  1.0051e+00]
[-1.4600e-02  3.0000e-04 -1.4900e-02  2.3000e-03  2.3000e-03  9.0000e-04
  1.0063e+00]
[-1.6600e-02  2.0000e-04 -1.7000e-02  2.0000e-03  3.1000e-03  1.1000e-03
  1.0068e+00]
[-1.8800e-02  1.0000e-04 -1.9300e-02  1.6000e-03  3.1000e-03  5.0000e-04
  1.0067e+00]
[-2.1400e-02 -1.0000e-04 -2.1600e-02  1.1000e-03  3.6000e-03  0.0000e+00
  1.0071e+00]
[-2.3500e-02 -2.0000e-04 -2.4200e-02  9.0000e-04  4.8000e-03  6.0000e-04
  1.0081e+00]
[-2.610e-02 -6.000e-04 -2.6

In [5]:
timestamp_query_list = [1305031097.6659, 1305031098.6600, 1305031098.6660,
                        1305031102.160407, 1305031102.862808, 1305031106.130445,
                        1305031117.607777, 1305031120.678467, 1305031128.722976,
                        1305031128.754646]

for timestamp in timestamp_query_list:
    # interpolated pose in trajectory format
    success, pose = interpolate_pose(gt_list, timestamps, timestamp)

    # interpolated pose in matrix format
    if success:
        pose_matrix = trajectory_to_matrix(pose)
        print(np.linalg.det(pose_matrix))

Timestamp out of range
Timestamp out of range
1.0000000000000004
0.9999999999999989
0.9999999999999994
1.0000000000000002
0.9999999999999999
1.0000000000000007
0.9999999999999994
0.9999999999999997


In [67]:
# Input: pose from trajectory, a 4x4 matrix representing RT4 = R4.dot(T4), first translate then rotate
# Output: a vector of [tx ty tz qx qy qz qw] to represent the pose
def matrix_to_trajectory(RT4):
    # 3x3 R
    R3 = RT4[:3,:3]

    # 3x1 RT
    RT3 = RT4[:3,3:4]

    # 3x1 T
    T3 = np.linalg.inv(R3).dot(RT3)

    trajectory = np.zeros(7)

    # Update translation
    trajectory[:3] = T3.reshape((3))

    # Update rotation
    trajectory[3:7] = R.from_matrix([R3]).as_quat().reshape(4)

    return trajectory

In [18]:
vbg = o3d.t.geometry.VoxelBlockGrid.load('./results/rgbd_dataset_freiburg1_xyz/TSDF_798.npz')
visualize_vbg_o3d(vbg, 180, 0, 0)

PointCloud on CPU:0 [823679 points (Float32)].
Attributes: normals (dtype = Float32, shape = {823679, 3}).


In [17]:
vbg = o3d.t.geometry.VoxelBlockGrid.load('./results/rgbd_dataset_freiburg3_cabinet/TSDF_300.npz')
visualize_vbg_o3d(vbg, 180, 0, 0)

PointCloud on CPU:0 [4201012 points (Float32)].
Attributes: normals (dtype = Float32, shape = {4201012, 3}).
